In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset, MFDataset
import matplotlib.pyplot as plt
import glob
from datetime import datetime, timedelta
from scipy import stats
from sklearn.metrics import mean_squared_error
import calendar

import sys, os, time, warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

In [5]:
def calc_vp_vpd(pa,ta,vpd):              # Specific humidity [kg/kg] from VPD
    eps = 0.622
    es = 611.2*np.exp(17.67*ta/(ta+243.5))   # Ta in ˚C
    eu = es - 100*vpd                   # VPD in hPa
    return eu # Pa in kPa

def calc_t_pbl(pbl):              # ERA5 PBL [m]
    lapse_rate = 6.5   # international standard atmosphere (ISA) with a temperature lapse rate ˚C/km
    return lapse_rate*pbl/1000+273.15 # K

def calc_p_pbl(ta,pa,pbl):           # Ta in ˚C, Pa in kPa, ERA5 PBL [m]
#     rd = 287.058
#     tmp = -9.8 / rd * ( np.log(calc_t_pbl(pbl)) - np.log(ta+273.15) ) + np.log( pa*1000. )
#     return np.exp(tmp) # Pa
    rd = 287.058
    tmp = -9.8 * ( np.log(calc_t_pbl(pbl)) - np.log(ta+273.15) ) * ( pbl - 2. ) / ( rd * ( calc_t_pbl(pbl) - (ta+273.15) ) )
    return pa*1000. * np.exp(tmp) # Pa

def calc_den(ta,pa,pbl):           # Ta in ˚C, Pa in kPa, ERA5 PBL [m]
    rd = 287.058
    pmean = ( calc_p_pbl(ta,pa,pbl) + pa*1000. ) / 2.
    tmean = ( calc_t_pbl(pbl) + (ta+273.15) ) / 2.
    return pmean / ( rd * tmean )

def cal_tci(x,y,qa,pa,ta,pbl): # Qs in kg/kg, Ta in ˚C, Pa in kPa, ERA5 PBL [m] 
    mask = (x.notnull() & y.notnull() & qa.notnull() & pa.notnull() & ta.notnull() & pbl.notnull())
    if mask.sum()>=10:
        cr = stats.pearsonr(x[mask], y[mask])[0]
        pval = stats.pearsonr(x[mask], y[mask])[1]
        xsd = stats.tstd(x[mask])
        ysd = stats.tstd(y[mask])
        xm = np.mean(x[mask])
        ym = np.mean(y[mask])
        me = np.mean( 2.5e6*qa[mask] ) # Moist energy [J/kg]
        te = np.mean( 1005. * (ta[mask]+273.15)*(100./pa[mask])**(0.286) ) # Thermal energy [J/kg]
        # print("density = {}".format(calc_den(ta,pa,pbl)))
        den = np.mean( calc_den(ta[mask],pa[mask],pbl[mask]) ) # kg/m3
    else:
        cr = np.nan
        pval = np.nan
        xsd = np.nan
        ysd = np.nan
        xm = np.nan
        ym = np.nan
        me = np.nan
        te = np.nan
        den = np.nan
    return cr*ysd, cr, xsd, ysd, xm, ym, me, te, den, pval

In [6]:
dir = './FULLSET/'
site_list = sorted(glob.glob(dir+'*_HR_CI_QC_PBLH.csv'))

######################################################################
f_start = 0 ; f_stop = len(site_list)     # These files/sites [206 stations]
######################################################################

data_save = ["TA_F","VPD_F","PA_F","LCL_DV","QA_DV","SWC_F_MDS_1","LE_F_MDS","H_F_MDS","EF_DV","NETRAD_DV"]

source_vars = ['SWC_F_MDS_1','SWC_F_MDS_1','SWC_F_MDS_1','SWC_F_MDS_1',
               'LE_F_MDS','EF_DV','H_F_MDS',
               'SWC_F_MDS_1',
               'LE_F_MDS','H_F_MDS','LE_F_MDS','H_F_MDS','EF_DV','NETRAD_DV',
               'QA_DV','TA_F']
target_vars = ['LE_F_MDS','H_F_MDS','EF_DV','NETRAD_DV',
               'QA_DV','QA_DV','TA_F',
               'LCL_DV',
               'PBLH_ERA5','PBLH_ERA5','LCL_DV','LCL_DV','LCL_DV','LCL_DV',
               'LCL_DV','LCL_DV']
pairings = ["SWC1_LE","SWC1_H","SWC1_EF","SWC1_NETRAD", # Land --> Heat flux
            "LE_QA","EF_QA","H_TA",                     # Heat flux --> Near-surface meteorology
            "SWC1_LCL",                                 # Land --> PBL characteristics
            "LE_PBLH","H_PBLH","LE_LCL","H_LCL","EF_LCL","NETRAD_LCL", # Heat flux --> PBL characteristics
            "QA_LCL","TA_LCL"]                                         #Near-surface meteorology --> PBL characteristics

month = [x for x in range(1,13)]
month_name = [calendar.month_abbr[x] for x in range(1,13)]

In [7]:
for s,site in enumerate(site_list[f_start:f_stop]):
    print (s,site.split("/")[8])
    
    # open hourly flux site observations
    f_siteid = site.split("/")[8].split("_")[0]      
    f1 = sorted(glob.glob(dir+f_siteid+'_HR_CI_QC_PBLH.csv'))
    df1 = pd.read_csv(f1[0],na_values=-9999)
    del f1
    time1 = pd.to_datetime(df1['time'])
    df1 = df1.drop(columns=['time'])
    df1['time'] = time1
    del time1
    
    # create daily 
    df2 = df1.resample('1D', on = 'time').mean()
    time1 = pd.to_datetime(df2.index)
    df2 = df2.set_index([ pd.Index([i for i in range(len(df2))]) ])
    df2['time'] = time1
    del time1
    
    # calculate soil moisture tendency
    dsdt = df2['SWC_F_MDS_1'].copy()
    dsdt[:] = np.nan
    for t in range(1,len(df2['SWC_F_MDS_1'])):
        dsdt[t] = df2['SWC_F_MDS_1'][t] - df2['SWC_F_MDS_1'][t-1]  

    th1 = np.nanstd(dsdt) * 2.
    mask = (dsdt>th1)
    mask1 = pd.Series(list(mask), dtype="bool", index=[x for x in range(1,len(mask)+1)])
    mask1[0] = False
    mask2 = mask1.sort_index(ascending=True)[:len(mask1)-1]
    mask3 = mask2 + mask
    
    # mask out rainfall days
    for col in data_save:
        df2[col] = np.ma.masked_where(mask,df2[col]) # pr day only maskout
        # df2[col] = np.ma.masked_where(mask3,df2[col]) # pr day + subsequent day maskout
        df2[col][df2[col].isnull()] = np.nan
        mis = df2[col][df2[col].isnull()].index
        for t in range(len(mis)):
            tloc = (df1['time'].dt.year == df2['time'].dt.year[mis[t]]) & (df1['time'].dt.month == df2['time'].dt.month[mis[t]]) & (df1['time'].dt.day == df2['time'].dt.day[mis[t]])
            df1[col][tloc] = np.nan
    del mask
    del mask1
    del mask2
    del mask3
    del dsdt
    
    # calculate LA coupling for each month to avoid seasonal cycle
    t_start = time.perf_counter()
    cnt1 = 0
    for m,mon in enumerate(month_name):
        cnt = 0
        # calculate TCI
        for p,par in enumerate(pairings):
            if source_vars[p] in df1.columns and target_vars[p] in df1.columns:
                mask2 = ( df2['time'].dt.month == month[m] )
                if np.nanstd(df2[source_vars[p]][mask2]) != 0. and np.nanstd(df2[target_vars[p]][mask2]) != 0.:
                    # daily mean TCI
                    tci, tci_r, tci_ssd, tci_tsd, xm, ym, me, te, den, pval = cal_tci(df2[source_vars[p]][mask2],df2[target_vars[p]][mask2],
                                                                                      df2['QA_DV'][mask2],df2['PA_F'][mask2],df2['TA_F'][mask2],
                                                                                      df2['PBLH_ERA5'][mask2])
                    if p==len(pairings)-1:
                        emask = (df2['time'].dt.month == month[m]) & (df2[source_vars[p]].notnull() & df2[target_vars[p]].notnull() & df2['QA_DV'].notnull() & df2['PA_F'].notnull() & df2['TA_F'].notnull() & df2['VPD_F'].notnull())
                        pbl = np.nanmean(df2['PBLH_ERA5'][emask])
                        del emask
                        d_out = pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval, me, te, den, pbl]]),
                                             columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval', 'ME_MEAN', 'TE_MEAN', 'DEN_MEAN', 'PBLH_MEAN'], index=['daily'])
                    else:
                        d_out = pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval]]),
                                             columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval'], index=['daily'])
                    # hourly mean TCI
                    for hr in range(24):
                        mask1 = ( df1['time'].dt.month == month[m] ) & ( df1['time'].dt.hour == hr )
                        tci, tci_r, tci_ssd, tci_tsd, xm, ym, me, te, den, pval = cal_tci(df1[source_vars[p]][mask1],df1[target_vars[p]][mask1],
                                                                                          df1['QA_DV'][mask1],df1['PA_F'][mask1],df1['TA_F'][mask1],
                                                                                          df1['PBLH_ERA5'][mask1])
                        
                        if p==len(pairings)-1:
                            emask = ( df1['time'].dt.month == month[m] ) & ( df1['time'].dt.hour == hr ) & (df1[source_vars[p]].notnull() & df1[target_vars[p]].notnull() & df1['QA_DV'].notnull() & df1['PA_F'].notnull() & df1['TA_F'].notnull() & df1['VPD_F'].notnull())
                            pbl = np.nanmean(df1['PBLH_ERA5'][emask])
                            del emask
                            d_out = d_out.append( pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval, me, te, den, pbl]]),
                                                               columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval', 'ME_MEAN', 'TE_MEAN', 'DEN_MEAN', 'PBLH_MEAN'], index=[str(hr)]) )
                        else:
                            d_out = d_out.append( pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval]]),
                                                               columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval'], index=[str(hr)]) )
                else:
                    tci = np.nan
                    tci_r = np.nan
                    tci_ssd = np.nan
                    tci_tsd = np.nan
                    xm = np.nan
                    ym = np.nan
                    me = np.nan
                    te = np.nan
                    den = np.nan
                    pbl = np.nan
                    pval = np.nan
                    if p==len(pairings)-1:
                        d_out = pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval, me, te, den, pbl]]),
                                             columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval', 'ME_MEAN', 'TE_MEAN', 'DEN_MEAN', 'PBLH_MEAN'], index=['daily'])
                    else:
                        d_out = pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval]]),
                                             columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval'], index=['daily'])
                    for hr in range(24):
                        if p==len(pairings)-1:
                            d_out = d_out.append( pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval, me, te, den, pbl]]),
                                                               columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval', 'ME_MEAN', 'TE_MEAN', 'DEN_MEAN', 'PBLH_MEAN'], index=[str(hr)]) )
                        else:
                            d_out = d_out.append( pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval]]),
                                                               columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval'], index=[str(hr)]) )
                
            else:
                tci = np.nan
                tci_r = np.nan
                tci_ssd = np.nan
                tci_tsd = np.nan
                xm = np.nan
                ym = np.nan
                me = np.nan
                te = np.nan
                den = np.nan
                pbl = np.nan
                pval = np.nan
                if p==len(pairings)-1:
                    d_out = pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval, me, te, den, pbl]]),
                                         columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval', 'ME_MEAN', 'TE_MEAN', 'DEN_MEAN', 'PBLH_MEAN'], index=['daily'])
                else:
                    d_out = pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval]]),
                                         columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval'], index=['daily'])
                    
                for hr in range(24):
                    if p==len(pairings)-1:
                        d_out = d_out.append( pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval, me, te, den, pbl]]),
                                                           columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval', 'ME_MEAN', 'TE_MEAN', 'DEN_MEAN', 'PBLH_MEAN'], index=[str(hr)]) )
                    else:
                        d_out = d_out.append( pd.DataFrame(np.array([[tci, tci_r, tci_ssd, tci_tsd, xm, ym, pval]]),
                                                           columns=[par, par+'_R', par+'_SSD', par+'_TSD', par+'_SM', par+'_TM', par+'_pval'], index=[str(hr)]) )
            
            if cnt==0:
                d_out1 = d_out
            else:
                d_out1 = pd.concat([d_out1, d_out], axis=1)
            cnt = cnt + 1
            del d_out
                    
                    
        d_out1['month'] = month[m]
        if cnt1==0:
            d_out2 = d_out1
        else:
            d_out2 = d_out2.append( d_out1 )
        cnt1 = cnt1 + 1
        del d_out1
    
    d_out2['sub-daily'] = d_out2.index
    d_out2.fillna(np.nan).to_csv("CI_mixing_diag_final_corr/"+f_siteid+"_CI_prmask_PBLH.csv",float_format='%.8g',index=False)
    print("end making Dataframe : ",time.perf_counter()-t_start,"seconds")
    del d_out2
    # sys.exit()

0 AR-SLu_HR_CI_QC_PBLH.csv
0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
          ...     
756    1751.611272
757    1290.623944
758    1491.332730
759    2330.321639
760    1516.406815
Length: 93, dtype: float64
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
756    22.776229
757    19.774229
758    27.167396
759    14.978354
760    15.093396
Name: VPD_F, Length: 93, dtype: float64


SystemExit: 